In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
playlist = pd.read_csv('data_ana/PLAYLIST_SEPARADO.csv', sep = ';')
junto = pd.read_csv('data_ana/junto2.csv', delimiter= ';',encoding='latin-1')
codigos = pd.read_csv('data_ana/codigos2.csv', sep = ';')
codigos_playlist = pd.read_csv('data_ana/GLOBALLY_FILTERED_MODIFIED_PLAYLIST.csv', sep = ';')

In [3]:
junto['PlaylistID'] = junto['PlaylistID'].apply(lambda x: x.replace('-', '_'))
playlist['PlaylistID'] = playlist['PlaylistID'].apply(lambda x: x.replace('-', '_'))
codigos_playlist['PlaylistID'] = codigos_playlist['PlaylistID'].apply(lambda x: x.replace('-', '_'))

In [4]:
codigos_playlist=codigos_playlist.iloc[:80,:]

In [5]:
junto=junto.merge(codigos_playlist,how='left',on=['PlaylistID'])

In [6]:
melted_df = playlist.melt(id_vars=['Group', 'ID_leaf', 'PlaylistID'], 
                    value_vars=['L1','L2','L3','L4','SIL','P1','SIL_1','P2','SIL_2','P3','SIL_3','P4','SIL_4','A1','SIL_5','A2','SIL_6','A3','SIL_7','A4'],
                    var_name='segmento', 
                    value_name='codigo_tratamiento')


In [7]:
segmentos = {'START' : 0,
            'L1' : 1.4,
            'L2' : 17.4,
            'L3' : 33.4,
            'L4' : 49.4,
            'SIL' : 65.4,
            'TRAN1' : 70.4,
            'P1' : 71.6,
            'SIL_1' : 87.6,
            'P2' : 103.6,
            'SIL_2' : 119.6,
            'P3' : 135.6,
            'SIL_3' : 151.6,
            'P4' : 167.6,
            'SIL_4' : 183.6,
            'TRAN2' : 199.6,
            'A1' : 200.8,
            'SIL_5' : 216.8,
            'A2' : 232.8,
            'SIL_6' : 248.8,
            'A3' : 264.8,
            'SIL_7' : 280.8,
            'A4' : 296.8,
            'END': 312.8
            }

segmentos_2 = {'START' : 0,
    'P1' : 1.4,
    'SIL_1' : 17.4,
    'P2' : 33.4,
    'SIL_2' : 49.4,
    'P3' : 65.4,
    'SIL_3' : 81.4,
    'P4' : 97.4,
    'SIL_4' : 113.4,
    'A1' : 129.4,
    'SIL_5' : 144.4,
    'A2' : 160.4,
    'SIL_6' : 176.4,
    'A3' : 192.4,
    'SIL_7' : 208.4,
    'A4' : 224.4,
    'END': 240.4
}

In [8]:
#saber en que tiempo empiza 
def find_segment(time, playlistid):
    if int(playlistid[-1]) == 1 or int(playlistid[-1]) == 2:
        previous_segment = 'START'
        for segment, value in segmentos.items():
            if time < value:
                return previous_segment
            previous_segment = segment

    previous_segment = 'START'
    for segment, value in segmentos_2.items():
        if time < value:
            return previous_segment
        previous_segment = segment
    return previous_segment

In [9]:
junto['segmento'] = junto.apply(lambda row: find_segment(row['Begin Time (s)'] - row['START'], row['PlaylistID']), axis = 1)

In [10]:
junto_tratamiento = junto.merge(melted_df, how = 'left', left_on=['PlaylistID', 'segmento'], right_on = ['PlaylistID','segmento'])

In [11]:
tratamientos = junto_tratamiento.merge(codigos, how = 'left', left_on='codigo_tratamiento', right_on = 'Tratamiento_codigo' )

In [12]:
columnas_tratamiento = ['Audio','TIPO', 'Group_x', 'ID_leaf_x', 'PlaylistID', 'Response', 'Tratamiento_significado','Delta Time (s)','SNR NIST Quick (dB)', 'segmento', 'Global_Code_LINEABASE',
       'Global_Code_PARENTESCO', 'Global_Code_ASOCIACION',   ]
tratamientos_columnas_interes = tratamientos[columnas_tratamiento]



In [13]:
def obtener_parentesco (row):
    if type(row['Tratamiento_significado']) == float:
        return row['Tratamiento_significado']
    if '-' in row['Tratamiento_significado']:
        parentesco = row['Tratamiento_significado'].split('-')[0]
        return parentesco
    return row['Tratamiento_significado']

def obtener_asociacion(row):
    if type(row['Tratamiento_significado']) == float:
        return row['Tratamiento_significado']
    if '-' in row['Tratamiento_significado']:
        parentesco = row['Tratamiento_significado'].split('-')[1]
        parentesco = parentesco.split('_')[0]
        return parentesco
    return row['Tratamiento_significado']

def obtener_riesgo(row):
    if type(row['Tratamiento_significado']) == float:
        return row['Tratamiento_significado']
    if '_' not in row['Tratamiento_significado']:

        return np.nan

    if '_' in row['Tratamiento_significado']:
        parentesco = row['Tratamiento_significado'].split('_')[-1]
        if parentesco in '1234567':
            return np.nan
        return parentesco
    return row['Tratamiento_significado']

    



In [14]:
tratamientos_columnas_interes['parentesco'] = tratamientos_columnas_interes.apply(obtener_parentesco, axis = 1 )
tratamientos_columnas_interes['asociacion'] = tratamientos_columnas_interes.apply(obtener_asociacion, axis = 1 )
tratamientos_columnas_interes['riesgo'] = tratamientos_columnas_interes.apply(obtener_riesgo, axis = 1 )

C:\Users\anaga\AppData\Local\Temp\ipykernel_26124\1591421628.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tratamientos_columnas_interes['parentesco'] = tratamientos_columnas_interes.apply(obtener_parentesco, axis = 1 )
C:\Users\anaga\AppData\Local\Temp\ipykernel_26124\1591421628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tratamientos_columnas_interes['asociacion'] = tratamientos_columnas_interes.apply(obtener_asociacion, axis = 1 )
C:\Users\anaga\AppData\Local\Temp\ipykernel_26124\1591421628

In [15]:
tratamientos_columnas_interes.to_csv('tratamientos_columnas_interes.csv', sep = ';')

In [3]:
tratamientos_columnas_interes = pd.read_csv('tratamientos_columnas_interes.csv', sep = ';')

In [8]:
final = pd.read_csv('TESIS_AMAG_OBJ2Y3_BD_FINAL.csv', sep = ';')

In [25]:
junto.loc[:,['ID_leaf','playlistID']]

KeyError: "None of [Index(['ID_leaf', 'playlistID'], dtype='object')] are in the [columns]"

In [9]:
playlistycodigos=melted_df.merge(codigos_playlist,how='left',on=['PlaylistID'])

In [11]:
playlist_completo=playlistycodigos.merge(codigos, how='left', left_on=['codigo_tratamiento'], right_on=['Tratamiento_codigo'])

In [22]:
final_completo[final_completo['ID_leaf'].isna()]

,Group_x,ID_leaf_x,PlaylistID,segmento,codigo_tratamiento,Group_y,ID_leaf_y,Global_Code_LINEABASE,Global_Code_PARENTESCO,Global_Code_ASOCIACION,Tratamiento_codigo,Tratamiento_significado_x,parentesco_x,asociacion_x,riesgo_x,ID_leaf,TIPO,Tratamiento_significado_y,parentesco_y,asociacion_y,riesgo_y,orden_segmento,Conteo
1,9,"9,82126E+14",Gr9_4300_1,L1,A,9,"9,82126E+14",2,42,78,A,P-A,P,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,13,"9,82126E+14",Gr13_8540_1,L1,D,13,"9,82126E+14",7,47,80,D,NP-NOAS,NP,NOAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,13,"9,82126E+14",Gr13_8521_1,L1,B,13,"9,82126E+14",8,48,81,B,P-NOAS,P,NOAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,7,"9,82126E+14",Gr7_5141_1,L1,C,7,"9,82126E+14",10,45,83,C,NP-A,NP,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,10,"9,82126E+14",Gr10_5921_1,L1,C,10,"9,82126E+14",16,54,87,C,NP-A,NP,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,14,"9,82126E+14",Gr14_6154_3,A4,J2,14,"9,82126E+14",40,57,80,J2,NP-NOAS_D,NP,NOAS,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1594,3,"9,002E+14",Gr3_9490_3,A4,I1,3,"9,002E+14",40,62,109,I1,P-A_D,P,A,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1595,3,"9,82126E+14",Gr3_4334_3,A4,J1,3,"9,82126E+14",40,43,108,J1,P-NOAS_D,P,NOAS,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1596,3,"9,82126E+14",Gr3_4346_3,A4,J1,3,"9,82126E+14",40,73,79,J1,P-NOAS_D,P,NOAS,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
final.dtypes

ID_leaf                     int64
PlaylistID                 object
TIPO                       object
Tratamiento_significado    object
parentesco                 object
asociacion                 object
riesgo                     object
segmento                   object
orden_segmento              int64
Conteo                      int64
dtype: object

In [14]:
playlist_completo['parentesco'] = playlist_completo.apply(obtener_parentesco, axis = 1 )
playlist_completo['asociacion'] = playlist_completo.apply(obtener_asociacion, axis = 1 )
playlist_completo['riesgo'] = playlist_completo.apply(obtener_riesgo, axis = 1 )

In [16]:
final_completo=playlist_completo.merge(final, how='left',on=['PlaylistID','segmento'])

In [17]:
final_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Group_x                    1600 non-null   int64  
 1   ID_leaf_x                  1600 non-null   object 
 2   PlaylistID                 1600 non-null   object 
 3   segmento                   1600 non-null   object 
 4   codigo_tratamiento         1600 non-null   object 
 5   Group_y                    1600 non-null   int64  
 6   ID_leaf_y                  1600 non-null   object 
 7   Global_Code_LINEABASE      1600 non-null   int64  
 8   Global_Code_PARENTESCO     1600 non-null   int64  
 9   Global_Code_ASOCIACION     1600 non-null   int64  
 10  Tratamiento_codigo         1495 non-null   object 
 11  Tratamiento_significado_x  1495 non-null   object 
 12  parentesco_x               1495 non-null   object 
 13  asociacion_x               1495 non-null   objec

In [ ]:
columnas_final = ['ID_leaf_x', 'PlaylistID','TIPO', 'Group_x','Tratamiento_significado','Delta Time (s)','SNR NIST Quick (dB)', 'segmento', 'Global_Code_LINEABASE',
       'Global_Code_PARENTESCO', 'Global_Code_ASOCIACION',   ]
tratamientos_columnas_interes = tratamientos[columnas_tratamiento]
